# Results 
- Plot training/test telemetry values, predictions, smoothed errors, and predicted and actual anomalies
- A specified results file from the `results` dir is used to highlight anomalous regions and read in data from the `data` dir

In [ ]:
import sys
import numpy as np
import os
sys.path.insert(0, '..')
from telemanom.plotting import Plotter
import pandas as pd
import plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode
import cufflinks as cf
import glob
cf.go_offline()
init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

## Select set of results to visualize

In [ ]:
# Default to most recent
results_fn = glob.glob('../results/*.csv')[-1]
print('Using most recent run: {}'.format(results_fn.split("/")[-1]))

run_id = results_fn.split("/")[-1][:-4]

## Parameters
- See `config.yaml` for parameter explanations

In [ ]:
with open('../data/{}/params.log'.format(run_id), 'r') as f:
    for i,row in enumerate(f.readlines()):
        if len(row) < 2:
            break
        if i > 0:
            print(' '.join(row.split(' ')[3:]).replace('\n',''))

## Summary of results

In [ ]:
plotter = Plotter(run_id)
plotter.all_result_summary()

## Interactive inline Plotly charts for viewing `y_test`, `y_hat`, and `smoothed errors (e_s)`
- **Blue** highlighted regions indicate anomalous sequences detected by the system
- If available, **Red** highlighted regions indicate true anomalous regions
- Can also optionally plot training data by setting `plot_train=True`

In [ ]:
from telemanom.helpers import Config
from telemanom.channel import Channel
from pathlib import Path
sys.path.insert(0, os.path.abspath('../../helpers'))
from helpers import (
    get_projects_and_run_settings,
    load_config,
    txt_to_npy_selective_reshape,
    plot_channel_hw
)

current_directory = Path.cwd()
parent_directory = current_directory.parent

PlotterHW = Plotter(run_id)
plotter.plot_channel = plot_channel_hw.__get__(PlotterHW)

config = Config('config.yaml')
prj_config = load_config('../../helpers/prj_config.yaml')
projects, run_settings = get_projects_and_run_settings(prj_config)

example_project = None
for project in projects:
    if run_settings['project'] in project:
        example_project = project[run_settings['project']]

if example_project is None:
    print(f"[anomaly_aie_lstm] Project name {run_settings['project']}' not found in prj_config")
else:
    for model_par in example_project['graphs']:
        for i in range(0, len(model_par['models'])):
            channel = Channel(config, model_par['models'][i])
            txt_file = os.path.join('../../lstmsettings['project'], 
                                    'output', 'hout_{}.txt'.format(model_par['models'][i]))
            npy_file = os.path.join(parent_directory, 'data', config.use_id, 'y_hat_hw', 
                                    '{}.npy'.format(model_par['models'][i]))
            txt_to_npy_selective_reshape(txt_file, npy_file, 16, (-1,))
            
            channel.y_hat = np.load(os.path.join(parent_directory, 'data', config.use_id, 
                                                 'y_hat_hw', '{}.npy'.format(model_par['models'][i])))
            channel.y_test = np.load(os.path.join(parent_directory, "data", 
                                                  "test", "{}.npy".format(model_par['models'][i])))

            channel.shape_data(channel.y_test, train=False)

            # Raw prediction error
            e = [abs(y_h-y_t[0]) for y_h, y_t in zip(channel.y_hat, channel.y_test)]
            smoothing_window = int(config.batch_size * config.window_size * config.smoothing_perc)

            # Create smoothed prediction error
            e_s = pd.DataFrame(e).ewm(span=smoothing_window).mean().values.flatten()
            e_s[:config.l_s] = [np.mean(e_s[:config.l_s * 2])] * config.l_s
            os.makedirs(os.path.join(parent_directory, 'data', config.use_id, 
                                     'smoothed_errors_hw'), exist_ok=True)
            np.save(os.path.join(parent_directory, 'data', config.use_id, 
                                 'smoothed_errors_hw', '{}.npy'.format(model_par['models'][i])), 
                                 np.array(e_s))

            plotter.plot_channel(model_par['models'][i], plot_errors=True, plot_train=False)
